# Ejercicio 01 (RESPUESTA)

## Objetivo
Crear un análisis integrado de campañas publicitarias combinando datos de Google Ads, Facebook Ads y Google Analytics, realizando un análisis predictivo simple y visualizando los resultados.


In [1]:
from facebook_business.adobjects.adaccount import AdAccount
from gaarf.report_fetcher import AdsReportFetcher
from facebook_business.api import FacebookAdsApi
from datetime import datetime, timedelta
from gaarf import GoogleAdsApiClient
from collections import defaultdict
from dotenv import load_dotenv
import plotly.express as px
from auth import ga_auth
import statsmodels
import polars as pl
import os

# Cargar variables de entorno
load_dotenv()

True

## 1. Configuración de Variables

In [2]:
# Fechas
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")

# Cuentas
google_ads_account = "7231659422"
facebook_ads_account = "1741832612768307"
analytics_property = "274438246"

## 2. Extracción de Datos de Google Ads

In [3]:
client = GoogleAdsApiClient(path_to_config="../google-ads.yaml")
report_fetcher = AdsReportFetcher(client)

query_text = f"""
SELECT
    segments.date,
    campaign.name,
    metrics.cost_micros,
    metrics.impressions,
    metrics.clicks
FROM campaign
WHERE segments.date BETWEEN '{start_date}' AND '{end_date}'
ORDER BY metrics.cost_micros DESC
"""

customer_ids = [google_ads_account]
report = report_fetcher.fetch(query_text, customer_ids)
df = pl.from_pandas(report.to_pandas())

In [4]:
df_google_ads = df.with_columns(
    [
        (pl.lit("google").alias("medio")),
        (pl.col("segments_date").str.to_date("%Y-%m-%d").alias("date")),
        (pl.col("campaign_name").alias("campaign")),
        ((pl.col("metrics_cost_micros") / 1000000).round(2).alias("cost")),
        (pl.col("metrics_impressions").alias("impressions")),
        (pl.col("metrics_clicks").alias("clicks"))
    ]
).select([
"date",
"medio",
"campaign",
"cost",
"impressions",
"clicks"
])

## 3. Extracción de Datos de Facebook Ads

In [5]:
FacebookAdsApi.init(
    access_token=os.getenv('FB_ACCESS_TOKEN')
)
fb_account = AdAccount(f"act_{facebook_ads_account}")
fields = [
    'campaign_name',
    'spend',
    'impressions',
    'clicks'
]
params = {
    'level': 'campaign',
    'time_range': {
        'since': start_date,
        'until': end_date
        },
    'time_increment': '1'
}

insights = fb_account.get_insights(
    fields=fields,
    params=params
)

df = pl.DataFrame([insight.export_all_data() for insight in insights])

In [6]:
df_facebook_ads = df.with_columns([
    (pl.lit("facebook").alias("medio")),
    (pl.col("date_start").str.to_date("%Y-%m-%d").alias("date")),
    (pl.col("campaign_name").alias("campaign")),
    (pl.col("spend").cast(pl.Float64).alias("cost")),
    (pl.col("impressions").cast(pl.Int64)),
    (pl.col("clicks").cast(pl.Int64))
]).select([
    "date",
    "medio",
    "campaign",
    "cost",
    "impressions",
    "clicks"
    ])

## 4. Extracción de Datos de Google Analytics

In [7]:
def ga_data(property_id, start_date, end_date, dimensions, metrics, filter_expr=None):
    analytics = ga_auth(['https://www.googleapis.com/auth/analytics.readonly'])
    request = {
        "requests": [{
            "dateRanges": [{
                "startDate": start_date,
                "endDate": end_date
            }],
            "dimensions": [{"name": dim} for dim in dimensions],
            "metrics": [{"name": met} for met in metrics],
            "limit": 250000
        }]
    }
    
    # Agregar filtro si se especifica
    if filter_expr:
        request["requests"][0]["dimensionFilter"] = {
            "filter": filter_expr
        }
    
    # Obtener datos
    response = analytics.properties().batchRunReports(
        property=f'properties/{property_id}', 
        body=request
    ).execute()
    
    # Procesar respuesta
    data = defaultdict(list)
    for row in response['reports'][0]['rows']:
        # Agregar dimensiones
        for dim, value in zip(dimensions, row['dimensionValues']):
            data[dim].append(value['value'])
        # Agregar métricas
        for met, value in zip(metrics, row['metricValues']):
            data[met].append(value['value'])
            
    return pl.DataFrame(data)


In [8]:
filtro_1 = {
    "fieldName": "sessionCampaignName",
    "stringFilter": {
        "matchType": "PARTIAL_REGEXP",
        "value": "^(aw|fb)(-|_)"
    }
}

df = ga_data(
    property_id=analytics_property,
    start_date=start_date,
    end_date=end_date,
    dimensions=['date', 'sessionCampaignName'],
    metrics=['sessions', 'transactions', 'purchaseRevenue'],
    filter_expr=filtro_1
)

In [9]:
df_analytics = df.with_columns(
    [
        (pl.col("date").str.to_date("%Y%m%d")),
        (pl.col("sessionCampaignName").alias("campaign")),
        (pl.col("sessions").cast(pl.Int64)),
        (pl.col("transactions").cast(pl.Int64)),
        (pl.col("purchaseRevenue").cast(pl.Float64).alias("revenue"))
    ]
).select([
    "date",
    "campaign",
    "sessions",
    "transactions",
    "revenue"
    ])

## 5. Integración de Datos
Combina los datos de las tres fuentes:

In [10]:
df_campaigns = pl.concat([df_google_ads, df_facebook_ads]).join(df_analytics, on=["date", "campaign"], how="left")
df_campaigns

date,medio,campaign,cost,impressions,clicks,sessions,transactions,revenue
date,str,str,f64,i64,i64,i64,i64,f64
2024-12-22,"""google""","""aw_innova_ecomm_do_pmax_all_ao""",52674.42,1066483,19170,16657,119,173391.100131
2024-12-17,"""google""","""aw_innova_ecomm_do_pmax_all_ao""",44426.45,944995,17222,15308,231,311068.949885
2024-12-18,"""google""","""aw_innova_ecomm_do_pmax_all_ao""",43715.51,1005570,17283,15692,222,285179.750024
2024-12-19,"""google""","""aw_innova_ecomm_do_pmax_all_ao""",43540.92,980042,17618,16055,191,270831.510332
2024-12-16,"""google""","""aw_innova_ecomm_do_pmax_all_ao""",43366.29,951268,19232,16497,252,344333.399813
…,…,…,…,…,…,…,…,…
2025-01-15,"""facebook""","""fb_innova_ecomm_think_auct_dab…",158.68,129309,6287,442,0,0.0
2025-01-15,"""facebook""","""fb_innova_ecomm_do_asc_ao_view""",180.83,331905,26124,2194,0,0.0
2025-01-15,"""facebook""","""fb_innova_ecomm_do_asc_ao_cart""",101.45,96410,6776,475,2,1139.000007


## 6. Análisis y Visualización
Crea visualizaciones para entender los datos:

In [11]:
df_campaigns.group_by("medio").agg([pl.sum("cost")]).plot.bar(x="cost", y="medio", color="medio").properties(title="Cost by medio")

alt.Chart(...)

In [12]:
df_campaigns.group_by("medio").agg([pl.sum("revenue")]).plot.bar(x="revenue", y="medio", color="medio").properties(title="Revenue by medio")

alt.Chart(...)

In [13]:
df_campaigns.group_by("date").agg([pl.sum("cost")]).plot.line(x="date", y="cost").properties(title="Cost trend")

alt.Chart(...)

In [14]:
df_campaigns.group_by("date").agg([pl.sum("revenue")]).plot.line(x="date", y="revenue").properties(title="Revenue trend")

alt.Chart(...)

In [19]:
px.scatter(df_campaigns, x="cost", y="revenue", trendline="ols")

In [16]:
df_campaigns_grouped = df_campaigns.group_by(["medio","campaign"]).agg([
        pl.sum("cost"),
        pl.sum("revenue")
    ])
px.scatter(df_campaigns_grouped, x="cost", y="revenue", facet_col="medio", color="medio", trendline="ols")

## Recursos Adicionales:
- Documentación de Polars: https://pola-rs.github.io/polars/
- Documentación de Plotly Express: https://plotly.com/python-api-reference/plotly.express.html
- Guía de GAQL: https://developers.google.com/google-ads/api/docs/query/overview